# Rock Paper Scissors

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import platform

print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

Python version: 3.7.6
Tensorflow version: 2.1.0
Keras version: 2.2.4-tf
